# Referral Outcomes

With this report we want to examine referral outcomes, in particular with regards to Sections 17 and 47 of the Children Act 1989:

## Section 17 (S17) of the Children Act 1989:

This section pertains to the provision of services to children in need, which includes their families and others.

Local authorities have a general duty to safeguard and promote the welfare of children within their area who are in need and to promote the upbringing of such children, wherever possible, by their families through providing a range of services appropriate to those children's needs.

"Children in need" are defined as children who:
a. Are unlikely to achieve or maintain a reasonable level of health or development without the provision of services; or
b. Their health or development is likely to be significantly impaired, or further impaired, without the provision of such services; or
c. Are disabled.

## Section 47 (S47) of the Children Act 1989:

This section relates to local authorities' duty to investigate situations where there is a reason to suspect that a child living in their area is suffering, or is likely to suffer, significant harm. "Significant harm" is a key concept in child protection and can encompass a wide range of adverse experiences, including neglect, physical, emotional, or sexual abuse.

Where they have reasonable cause to suspect that a child who lives or is found in their area is suffering or likely to suffer significant harm, the authority is required to make inquiries (or cause inquiries to be made) to decide whether they should take action to safeguard or promote the child's welfare.

These inquiries may lead to a child protection conference, where professionals come together to discuss concerns and decide on future actions, which may include a child protection plan.

In the CIN Census data (abbreviated to only the fields we consider here for clarity), this will look like:
```
<CINdetails>
    <CINreferralDate>1970-10-06</CINreferralDate>
    <Assessments>
        <AssessmentActualStartDate>1970-06-03</AssessmentActualStartDate>
    </Assessments>
    <Section47>
        <S47ActualStartDate>1970-06-02</S47ActualStartDate>
    </Section47>
</CINdetails
```

The "flatfile" ingest we provide of this report, creates an "event" based log of this file (again we have left out columns for clarity):

| LAchildID   | Date       | Type                        | CINreferralDate   | AssessmentActualStartDate   | CPPstartDate   |
|:------------|:-----------|:----------------------------|:------------------|:----------------------------|:---------------|
| DfEX0000001 | 1970-10-06 | CINreferralDate             | 1970-10-06        | nan                         | nan            |
| DfEX0000001 | 1971-02-27 | CINclosureDate              | 1970-10-06        | nan                         | nan            |
| DfEX0000001 | 1970-06-03 | AssessmentActualStartDate   | 1970-10-06        | 1970-06-03                  | nan            |
| DfEX0000001 | 1971-07-18 | AssessmentAuthorisationDate | 1970-10-06        | 1970-06-03                  | nan            |
| DfEX0000001 | 1971-01-24 | CINPlanStartDate            | 1970-10-06        | nan                         | nan            |
| DfEX0000001 | 1971-01-26 | CINPlanEndDate              | 1970-10-06        | nan                         | nan            |
| DfEX0000001 | 1970-06-02 | S47ActualStartDate          | 1970-10-06        | nan                         | nan            |
| DfEX0000001 | 1970-02-17 | CPPstartDate                | 1970-10-06        | nan                         | 1970-02-17     |
| DfEX0000001 | 1971-03-14 | CPPendDate                  | 1970-10-06        | nan                         | 1970-02-17     |
| DfEX0000001 | 1971-02-15 | CPPreviewDate               | 1970-10-06        | nan                         | 1970-02-17     |

With this report we want to clarify the length it takes from referral to a child assessment and/or S47 inquiry. 

In [1]:
import pandas as pd
import numpy as np
import io

We start by loading some sample data

In [2]:
data = """
LAchildID,Date,Type,CINreferralDate,ReferralSource,PrimaryNeedCode,CINclosureDate,ReasonForClosure,DateOfInitialCPC,ReferralNFA,CINPlanStartDate,CINPlanEndDate,S47ActualStartDate,InitialCPCtarget,ICPCnotRequired,AssessmentActualStartDate,AssessmentInternalReviewDate,AssessmentAuthorisationDate,Factors,CPPstartDate,CPPendDate,InitialCategoryOfAbuse,LatestCategoryOfAbuse,NumberOfPreviousCPP,UPN,FormerUPN,UPNunknown,PersonBirthDate,ExpectedPersonBirthDate,GenderCurrent,PersonDeathDate,PersonSchoolYear,Ethnicity,Disabilities
DfEX0000001,1970-10-06,CINreferralDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,,,,,,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-02-27,CINclosureDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,,,,,,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1970-06-03,AssessmentActualStartDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,1970-06-03,1970-06-22,1971-07-18,"2A,2B",,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-07-18,AssessmentAuthorisationDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,1970-06-03,1970-06-22,1971-07-18,"2A,2B",,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-01-24,CINPlanStartDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,1971-01-24,1971-01-26,,,,,,,,,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-01-26,CINPlanEndDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,1971-01-24,1971-01-26,,,,,,,,,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1970-06-02,S47ActualStartDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-06-17,0,,,1970-06-02,1970-06-23,0,,,,,,,,,,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1970-02-17,CPPstartDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,,,,,1970-02-17,1971-03-14,PHY,PHY,10,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-03-14,CPPendDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,,,,,1970-02-17,1971-03-14,PHY,PHY,10,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
DfEX0000001,1971-02-15,CPPreviewDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,,,,,,,,,,1970-02-17,1971-03-14,PHY,PHY,10,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,,WBRI,"HAND,HEAR"
""".strip()
data = pd.read_csv(io.StringIO(data), parse_dates=[
    'Date', 'CINreferralDate', 'CINclosureDate', 'DateOfInitialCPC', 
    'CINPlanStartDate', 'CINPlanEndDate', 'S47ActualStartDate', 'InitialCPCtarget',
    'AssessmentActualStartDate', 'AssessmentInternalReviewDate', 'AssessmentAuthorisationDate',
    'CPPstartDate', 'CPPendDate',
    'PersonBirthDate', 'ExpectedPersonBirthDate', 'PersonDeathDate'
])
data.head(n=1)

,LAchildID,Date,Type,CINreferralDate,ReferralSource,PrimaryNeedCode,CINclosureDate,ReasonForClosure,DateOfInitialCPC,ReferralNFA,...,UPN,FormerUPN,UPNunknown,PersonBirthDate,ExpectedPersonBirthDate,GenderCurrent,PersonDeathDate,PersonSchoolYear,Ethnicity,Disabilities
0,DfEX0000001,1970-10-06,CINreferralDate,1970-10-06,1A,N4,1971-02-27,RC1,1970-12-06,0,...,A123456789123,X98765432123B,UN3,1966-03-24,1966-03-22,1,1980-10-08,NaN,WBRI,"HAND,HEAR"


So based on the big "wide" format, we can narrow things down to the few columns we are interested in. In terms of reports, each referral consists of a unique combination of LAchildID and CINreferralDate, and below that there will be a unique AssessmentActualStartDate for S17 Assessment or S47ActualStartDate for S47 Inquiry.

Starting with S17, let's find the unique combinations of those fields, and calculate the duration between referral and assessment:

In [3]:
s17_dates = data[data["AssessmentActualStartDate"].notna()][["LAchildID", "CINreferralDate", "AssessmentActualStartDate"]].drop_duplicates()
s17_dates["days_to_s17"] = s17_dates["CINreferralDate"] - s17_dates["AssessmentActualStartDate"]
s17_dates["days_to_s17"] = s17_dates["days_to_s17"].dt.days

# Remove any that are less than zero - it shouldn't happen, but just in case
s17_dates = s17_dates[s17_dates["days_to_s17"] >= 0]

s17_dates

,LAchildID,CINreferralDate,AssessmentActualStartDate,days_to_s17
2,DfEX0000001,1970-10-06,1970-06-03,125


We can do exactly the same for S47:

In [4]:
s47_dates = data[data["S47ActualStartDate"].notna()][["LAchildID", "CINreferralDate", "S47ActualStartDate"]].drop_duplicates()
s47_dates["days_to_s47"] = s47_dates["CINreferralDate"] - s47_dates["S47ActualStartDate"]
s47_dates["days_to_s47"] = s47_dates["days_to_s47"].dt.days

# Remove any that are less than zero - it shouldn't happen, but just in case
s47_dates = s47_dates[s47_dates["days_to_s47"] >= 0]

s47_dates

,LAchildID,CINreferralDate,S47ActualStartDate,days_to_s47
6,DfEX0000001,1970-10-06,1970-06-02,126


We can now merge these back with the CIN record. Since we want to see referrals that led to neither S17 or S47, we create a unique view of all referrals as a base to merge the others into:

In [5]:
merged = data[["LAchildID", "CINreferralDate"]].drop_duplicates()
merged = merged.merge(s17_dates, how="left", on=["LAchildID", "CINreferralDate"])
merged = merged.merge(s47_dates, how="left", on=["LAchildID", "CINreferralDate"])
merged

,LAchildID,CINreferralDate,AssessmentActualStartDate,days_to_s17,S47ActualStartDate,days_to_s47
0,DfEX0000001,1970-10-06,1970-06-03,125,1970-06-02,126


Finally, we add a "referral_outcome" giving us one of NFA, S17, S47 or BOTH depending on which records were found. 

In [6]:
neither = merged['AssessmentActualStartDate'].isna() & merged['S47ActualStartDate'].isna()
s17_set = merged['AssessmentActualStartDate'].notna() & merged['S47ActualStartDate'].isna()
s47_set = merged['AssessmentActualStartDate'].isna() & merged['S47ActualStartDate'].notna()
both_set = merged['AssessmentActualStartDate'].notna() & merged['S47ActualStartDate'].notna()

merged['referral_outcome'] = np.select([neither, s17_set, s47_set, both_set], ['NFA', 'S17', 'S47', 'BOTH'], default=None)
merged

,LAchildID,CINreferralDate,AssessmentActualStartDate,days_to_s17,S47ActualStartDate,days_to_s47,referral_outcome
0,DfEX0000001,1970-10-06,1970-06-03,125,1970-06-02,126,BOTH


We can import and run this as a report:

In [8]:
from liiatools.cin_census_pipeline.reports import referral_outcomes

referral_outcomes(data)

,LAchildID,CINreferralDate,AssessmentActualStartDate,days_to_s17,S47ActualStartDate,days_to_s47,referral_outcome
0,DfEX0000001,1970-10-06,1970-06-03,125,1970-06-02,126,BOTH
